## Indian Premier League (IPL) Regression Analysis

In [19]:
# %%capture
# # Due to the configuration of the base Jupter image, the following imports are required for the regressions in the assignment to report the correct metrics

# import sys 
# !{sys.executable} -m pip uninstall statsmodels --yes 
# !{sys.executable} -m pip uninstall numpy --yes
# !{sys.executable} -m pip uninstall pandas --yes 
# !{sys.executable} -m pip uninstall patsy --yes 
# !{sys.executable} -m pip install numpy==1.17
# !{sys.executable} -m pip install pandas==1.0
# !{sys.executable} -m pip install patsy==0.5.2
# !{sys.executable} -m pip install statsmodels==0.11.1

In [20]:
# As usual, we begin by loading the packages we will need

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import statsmodels.formula.api as smf

## Step 1
Load the data

In [21]:
# Now we load the data

IPL=pd.read_excel("Assignment Data/IPL (assignment) data.xlsx")

IPL.head()

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000


## Step 2
Create the sum of salaries in each season

In [22]:
all_sals = (IPL.groupby('year')
               .sum()
               .rename({'salaries':'all_sals'},axis=1)
               .loc[:,'all_sals']
               .to_frame()
               .reset_index()
           )
all_sals.head()

,year,all_sals
0,2008,34105000
1,2009,33445000
2,2010,33075000
3,2011,62210000
4,2012,59706250


## Step 3
Create a variable for team salary divided by total salaries for that season (relsal)

In [23]:
# First merge
IPL = IPL.merge(all_sals, on='year', how='left')

# Then perform relative salary column
IPL['relsal'] = IPL['salaries'] / IPL['all_sals']

IPL.head()

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,all_sals,relsal
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000,0.170796
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000,0.202272
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000,0.147846
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000,0.101752
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250,0.132314


## Step 4
Create a value for win percentage. Define win pct as $wins/games$

In [24]:
IPL['wpct'] = IPL['won'] / IPL['played']

## Step 5
Create the lagged value of win pct for each team

In [26]:
IPL['lag_wpct'] = IPL.groupby('team')['wpct'].shift(1)
IPL.head()

,year,team,played,won,lost,noresult,points,netrunrate,champions,runnersup,third,fourth,salaries,all_sals,relsal,wpct,lag_wpct
0,2008,Chennai Super Kings,14,8,6,0,16,-0.192,0,1,0,0,5825000,34105000,0.170796,0.571429,NaN
1,2009,Chennai Super Kings,14,8,5,1,17,0.951,0,0,0,0,6765000,33445000,0.202272,0.571429,0.571429
2,2010,Chennai Super Kings,14,7,7,0,14,0.274,1,0,0,0,4890000,33075000,0.147846,0.500000,0.571429
3,2011,Chennai Super Kings,14,9,5,0,18,0.443,1,0,0,0,6330000,62210000,0.101752,0.642857,0.500000
4,2012,Chennai Super Kings,16,8,7,1,17,0.100,0,1,0,0,7900000,59706250,0.132314,0.500000,0.642857


## Step 6
Regress win pct on:
 - relsal
 - relsal + lagged win pct
 - relsal + lagged win pct + team fixed effects

In [29]:
# relsal
reg1 = smf.ols(formula='wpct ~ relsal', data=IPL).fit()
reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wpct   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.010
Method:                 Least Squares   F-statistic:                    0.1428
Date:                Wed, 01 Dec 2021   Prob (F-statistic):              0.706
Time:                        13:32:28   Log-Likelihood:                 45.280
No. Observations:                  92   AIC:                            -86.56
Df Residuals:                      90   BIC:                            -81.52
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.4687      0.063      7.461      0.000       0.344       0.593
relsal         0.1923      0.509      0.378      0.706      -0.819       1.203
==============================================================================
Omnibus:                        2.962   Durbin-Watson:                   1.995
Prob(Omnibus):                  0.227   Jarque-Bera (JB):                2.958
Skew:                          -0.410   Prob(JB):                        0.228
Kurtosis:                       2.683   Cond. No.                         33.1
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# relsal + lag_wpct
reg2 = smf.ols(formula='wpct ~ relsal + lag_wpct', data=IPL).fit()
reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wpct   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.374
Date:                Wed, 01 Dec 2021   Prob (F-statistic):              0.259
Time:                        13:33:05   Log-Likelihood:                 43.331
No. Observations:                  79   AIC:                            -80.66
Df Residuals:                      76   BIC:                            -73.55
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3527      0.086      4.088      0.000       0.181       0.525
relsal         0.6561      0.523      1.255      0.213      -0.385       1.697
lag_wpct       0.1245      0.108      1.152      0.253      -0.091       0.340
==============================================================================
Omnibus:                        3.120   Durbin-Watson:                   1.832
Prob(Omnibus):                  0.210   Jarque-Bera (JB):                2.878
Skew:                          -0.466   Prob(JB):                        0.237
Kurtosis:                       2.924   Cond. No.                         37.0
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
# relsal + lag_wpct + C(team)
reg3 = smf.ols(formula='wpct ~ relsal + lag_wpct + C(team)', data=IPL).fit()
reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   wpct   R-squared:                       0.290
Model:                            OLS   Adj. R-squared:                  0.148
Method:                 Least Squares   F-statistic:                     2.046
Date:                Wed, 01 Dec 2021   Prob (F-statistic):             0.0303
Time:                        13:33:15   Log-Likelihood:                 55.477
No. Observations:                  79   AIC:                            -82.95
Df Residuals:                      65   BIC:                            -49.78
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==========================================================================================================
                                             coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------
Intercept                                  0.6720      0.138      4.887      0.000       0.397       0.947
C(team)[T.Deccan Chargers]                -0.1830      0.085     -2.160      0.034      -0.352      -0.014
C(team)[T.Delhi Daredevils]               -0.2050      0.069     -2.955      0.004      -0.344      -0.066
C(team)[T.Gujarat Lions]                  -0.3204      0.143     -2.242      0.028      -0.606      -0.035
C(team)[T.Kings XI Punjab]                -0.1677      0.068     -2.451      0.017      -0.304      -0.031
C(team)[T.Kochi Tuskers Kerala]         3.026e-16   4.33e-16      0.699      0.487   -5.62e-16    1.17e-15
C(team)[T.Kolkata Knight Riders]          -0.0997      0.065     -1.529      0.131      -0.230       0.031
C(team)[T.Mumbai Indians]                 -0.0345      0.065     -0.530      0.598      -0.165       0.096
C(team)[T.Pune Warriors India]            -0.3870      0.118     -3.275      0.002      -0.623      -0.151
C(team)[T.Rajasthan Royals]               -0.1403      0.086     -1.632      0.107      -0.312       0.031
C(team)[T.Rising Pune Supergiants]         0.0177      0.144      0.123      0.902      -0.269       0.305
C(team)[T.Royal Challengers Bangalore]    -0.1284      0.064     -1.999      0.050      -0.257    -8.9e-05
C(team)[T.Sunrisers Hyderabad]            -0.0676      0.076     -0.883      0.380      -0.220       0.085
relsal                                    -0.1494      0.691     -0.216      0.829      -1.529       1.230
lag_wpct                                  -0.0786      0.115     -0.683      0.497      -0.308       0.151
==============================================================================
Omnibus:                        2.559   Durbin-Watson:                   2.206
Prob(Omnibus):                  0.278   Jarque-Bera (JB):                1.997
Skew:                          -0.172   Prob(JB):                        0.368
Kurtosis:                       3.699   Cond. No.                     2.01e+16
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.68e-31. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""